In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset
from skimage import io
import torchvision
import torchvision.transforms as transform
import os
import cv2 as cv
from PIL import Image
from torch.utils.data import DataLoader
from tqdm import tqdm

# from utils import (
#     load_checkpoints,
#     save_checkpoints,
#     get_loader,
#     check_accuracy,
#     save_predictions_as_imgs
# )
from torch.utils.data import random_split


import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
from pipe_dataset import pipedataset
from utils import (
    load_checkpoint,
    save_checkpoints,
    get_loaders,
    check_accuracy,
    #save_predictions_as_imgs
)
from model_bulding import deeplabv3

In [ ]:
device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')

In [ ]:
#Hyperparameters:
Lr_rt = 1e-4
batch_size=32
epochs = 3
img_hight = 320
img_width = 240
PIN_MEMORY = True
LOAD_MODEL = True


## make data loader


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
img_dir = "/content/drive/MyDrive/pipe_data_set/small_p_data"
mask_dir = "/content/drive/MyDrive/pipe_data_set/merge_pipedata_label"

In [ ]:
data_set = pipedataset(img_dir, mask_dir, transform=None)

In [ ]:
generator = torch.Generator().manual_seed(42)
train_ds, val_ds, test_ds = random_split(data_set, [0.7, 0.2, 0.1], generator=generator)

In [ ]:
train_loader = DataLoader(dataset=train_ds, batch_size = 32, shuffle=True)
val_loader = DataLoader(dataset=val_ds, batch_size=32, shuffle=False)

In [ ]:
class diceloss(torch.nn.Module):
    def init(self):
        super(diceloss, self).init()
    def forward(self,pred, target):
       smooth = 1.
       iflat = pred.contiguous().view(-1)
       tflat = target.contiguous().view(-1)
       intersection = (iflat * tflat).sum()
       A_sum = torch.sum(iflat * iflat)
       B_sum = torch.sum(tflat * tflat)
       return 1 - ((2. * intersection + smooth) / (A_sum + B_sum + smooth) )

In [ ]:
def  train_fn(loader, model, optimizer, loss_fn, scaler):
  loop = tqdm(loader)
  for batch_idx, (data,targets) in enumerate(loop):
    data = data.float().to(device=device)
    data = data.permute(0,3,1,2)
    targets = targets.float().unsqueeze(1).to(device=device)
    #print("target", targets.size())
    #targets = targets.permute(0,3,1,2)

    with torch.cuda.amp.autocast():
      pred = model(data)
      loss = loss_fn(pred, targets)

    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    loop.set_postfix(loss=loss.item())




In [ ]:
def main():
  model = deeplabv3(in_cha=3).to(device)
  loss_fn = diceloss()#nn.BCEWithLogitsLoss()
  optimizer = optim.Adam(model.parameters(), lr = Lr_rt)

  #train_loader, val_loader = get_loaders(img_dir, mask_dir, batch_size, num_workers=4, pin_memory=True)
  scaler = torch.cuda.amp.GradScaler()
  for epoch in range(epochs):
    train_fn(train_loader, model, optimizer, loss_fn, scaler)
    check_accuracy(val_loader, model, device=device)

In [ ]:
  main()

  0%|          | 1/269 [00:43<3:14:14, 43.49s/it, loss=1]

torch.Size([32, 48, 121, 161])
c_1 torch.Size([32, 16, 124, 164])
torch.Size([32, 32, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41])
y_1 torch.Size([32, 32, 31, 41])


  1%|          | 2/269 [01:27<3:14:57, 43.81s/it, loss=1]

torch.Size([32, 48, 121, 161])
c_1 torch.Size([32, 16, 124, 164])
torch.Size([32, 32, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41])
y_1 torch.Size([32, 32, 31, 41])


  1%|          | 3/269 [02:16<3:24:18, 46.09s/it, loss=1]

...

torch.Size([32, 48, 121, 161])
c_1 torch.Size([32, 16, 124, 164])
torch.Size([32, 32, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41])
y_1 torch.Size([32, 32, 31, 41])


 72%|███████▏  | 195/269 [2:20:36<55:36, 45.08s/it, loss=0.939]

torch.Size([32, 48, 121, 161])
c_1 torch.Size([32, 16, 124, 164])
torch.Size([32, 32, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41])
y_1 torch.Size([32, 32, 31, 41])


 73%|███████▎  | 196/269 [2:21:21<54:46, 45.02s/it, loss=0.937]

torch.Size([32, 48, 121, 161])
c_1 torch.Size([32, 16, 124, 164])
torch.Size([32, 32, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41]) torch.Size([32, 8, 31, 41])
y_1 torch.Size([32, 32, 31, 41])
